In [0]:
!pip install yfinance

In [0]:
spark.catalog.setCurrentDatabase("default")

In [0]:
import yfinance as yf
import pandas as pd
from datetime import datetime,timedelta

In [0]:
end=datetime.now().date()
start=(datetime.now()-timedelta(days=1095)).date()
sw=yf.download("SWARAJENG.NS",start=start,end=end)
sw=sw.reset_index()
sw.head(5)
sw.columns=[col[0] if isinstance(col,tuple) else col for col in sw.columns]
sw=sw[['Date','Close']].copy()
sw["Date"]=sw["Date"].dt.date
sw["Close"]=sw["Close"].round(2)
sw.rename(columns={'Close':'SwarajEng_Close'},inplace=True)
sw.sort_values(by='Date',ascending=False).head(5)

In [0]:
auto=yf.download("^CNXAUTO",start=start,end=end)
auto=auto.reset_index()
auto.columns=[col[0] if isinstance(col,tuple) else col for col in auto.columns]
auto=auto[['Date','Close']].copy()
auto["Date"]=auto["Date"].dt.date
auto["Close"]=auto["Close"].round(2)
auto.rename(columns={'Close':'CNXAUTO_Close'},inplace=True)
auto.sort_values(by="Date",ascending=False).head(10)

In [0]:
Nifty=yf.download("^NSEI",start=start,end=end)
Nifty.head(5)
Nifty=Nifty.reset_index()
Nifty.columns=[col[0] if isinstance(col,tuple) else col for col in Nifty.columns]
Nifty=Nifty[['Date','Close']].copy()
Nifty["Date"]=Nifty["Date"].dt.date
Nifty["Close"]=Nifty["Close"].round(2)
Nifty.rename(columns={'Close':'Nifty_Close'},inplace=True)
Nifty.sort_values(by="Date",ascending=False).head(10)

In [0]:
merge=pd.merge(sw,auto,on='Date',how='outer')
merge=pd.merge(merge,Nifty,on='Date',how='outer')
merge=merge.sort_values(by='Date')
merge=merge.ffill()
merged=spark.createDataFrame(merge)
merged=merged.write.option("overwriteSchema","true").mode("overwrite").format("delta").saveAsTable("sw_indices_merged")

In [0]:
%sql

select * from sw_indices_merged
limit 10;
